In [6]:
from openfisca_us import IndividualSim
import plotly.express as px
import pandas as pd

qbi_df = pd.DataFrame()
for w2_wages in range(0, 100_000, 20_000):
    sim = IndividualSim()
    sim.add_person(age=26, w2_wages_from_qualified_business=w2_wages)
    sim.add_tax_unit(members=["person_1"])
    sim.vary("self_employment_income", min=0, max=300_000)
    df = pd.DataFrame({
        "W2 wages": w2_wages,
        "Qualified business income": sim.calc("qualified_business_income")[0],
        "QBI deduction": sim.calc("qualified_business_income_deduction")[0],
    })
    qbi_df = pd.concat([qbi_df, df])

In [9]:
px.line(qbi_df, color="W2 wages", x="Qualified business income", y="QBI deduction").update_layout(
    title="QBI deduction by business income and W2 wage examples",
    width=800,
    height=600,
    xaxis_tickformat="$,.0f",
    yaxis_tickformat="$,.0f",
)

In [24]:
from openfisca_core.simulation_builder import SimulationBuilder
from openfisca_us import CountryTaxBenefitSystem

WITH_AXES = {
    "people": {
        "person": {
            "age": {"2022": 18}
        },
    },
    "tax_units": {
        "tax_unit": {
            "members": ["person"],
        }
    },
    "spm_units": {
        "spm_unit": {
            "members": ["person"],
        }
    },
    "households": {
        "household": {
            "members": ["person"],
        }
    },
    "families": {
        "family": {
            "members": ["person"],
        }
    },
    'axes': [
        [{'count':100, 'name':'w2_wages_from_qualified_business', 'min':0, 'max':200_000, 'period':'2022'}],
        [{'count':100, 'name':'self_employment_income', 'min':0, 'max':200_000, 'period':'2022'}]
    ]
    }

simulation_builder = SimulationBuilder()
simulation = simulation_builder.build_from_entities(CountryTaxBenefitSystem(), WITH_AXES)

import plotly.graph_objects as go

income = simulation.calculate('self_employment_income', 2022).reshape(100, 100)
w2_wages = simulation.calculate('w2_wages_from_qualified_business', 2022).reshape(100, 100)
qbid = simulation.calculate('qualified_business_income_deduction', 2022).reshape(100, 100)
# data is now a 100x100 array of self_employment_income values
# we plot a surface plot of the data

fig = go.Figure(data=go.Surface(x=income, y=w2_wages, z=qbid))
fig.update_layout(
    title="QBI deduction (z) by business income (x) and W2 wages (y)",
    width=800,
    height=600,
    xaxis_tickformat="$,.0f",
    yaxis_tickformat="$,.0f",
)